# Greeks Validation Notebook

This notebook is for validation and case studies:
- Validate greeks calculations against textbook behavior
- Test hedge effectiveness
- Run scenario analysis

In [2]:
import sys
import os

# Add src directory to Python path
# Get the project root (assumes notebook is run from project root or notebooks directory)
current_dir = os.getcwd()
if os.path.basename(current_dir) == 'notebooks':
    project_root = os.path.dirname(current_dir)
else:
    project_root = current_dir

src_path = os.path.join(project_root, 'src')
if os.path.exists(src_path) and src_path not in sys.path:
    sys.path.insert(0, src_path)

import pandas as pd
import numpy as np
from data_loader import DataLoader
from greeks_calculator import GreeksCalculator
from portfolio_aggregator import PortfolioAggregator
from hedge_optimizer import HedgeOptimizer

In [3]:
data_loader = DataLoader()
data_loader.load_all_data(symbols=["AAPL", "GOOG", "MSFT", "AMZN", "TSLA"])

Loading all required data
Using cached stock data for 5 symbols
Saved market_data.csv (5 rows)
Fetching risk-free rates
Using yfinance fallback for Treasury rates
Fetched 9 risk-free rate tenors
Saved rates.csv (9 rows)
Building volatility surface for 5 symbols
Using cached stock data for 5 symbols
Fetching options chain for AAPL
Fetched 2360 options for AAPL
Fetching options chain for GOOG
Fetched 2255 options for GOOG
Fetching options chain for MSFT
Fetched 2829 options for MSFT
Fetching options chain for AMZN
Fetched 2014 options for AMZN
Fetching options chain for TSLA
Fetched 4998 options for TSLA
Built volatility surface with 11646 points
Saved vol_surface.csv (11646 rows)
Generating 20 synthetic positions
Using cached stock data for 5 symbols
Saved positions.csv (20 rows)
Generated 20 positions
Data loading complete


{'market_data':   symbol  spot_price  dividend_yield  borrow_cost_bps  \
 0   AAPL      258.21            0.40           210.00   
 1   GOOG      333.16            0.25            16.01   
 2   MSFT      456.66            0.80            19.42   
 3   AMZN      238.18            0.00            21.77   
 4   TSLA      438.57            0.00            26.69   
 
                  last_updated  
 0  2026-01-16T14:37:24.206316  
 1  2026-01-16T14:37:24.206316  
 2  2026-01-16T14:37:24.206316  
 3  2026-01-16T14:37:24.206316  
 4  2026-01-16T14:37:24.206316  ,
 'rates':    tenor_days     rate
 0          30  0.03565
 1          90  0.03565
 2         180  0.03565
 3         365  0.03565
 4         730  0.03762
 5        1095  0.03762
 6        1825  0.03762
 7        2555  0.03762
 8        3650  0.04160,
 'vol_surface':       symbol      expiry  strike  moneyness  implied_vol
 0       AAPL  2026-01-16   130.0   0.503466     0.000010
 68      AAPL  2026-01-16   130.0   0.503466     0.5000

In [4]:
greeks_calculator = GreeksCalculator()
greeks_calculator.run_pipeline()

,position_id,symbol,quantity,instrument_type,strike,expiry,option_type,spot_price,dividend_yield,borrow_cost_bps,...,delta,gamma,vega,theta,rho,position_delta,position_gamma,position_vega,position_theta,position_rho
0,POS_0001,AMZN,131,equity,NaN,NaN,NaN,238.18,0.00,21.77,...,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.310000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,POS_0002,MSFT,-816,equity,NaN,NaN,NaN,456.66,0.80,19.42,...,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-8.160000e+02,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
2,POS_0003,MSFT,-28,option,468.75,2026-05-01,put,456.66,0.80,19.42,...,-9.854979e-01,1.103023e-02,8.967103e+00,1.609128e+01,-1.303297e+02,2.759394e+01,-3.088464e-01,-2.510789e+02,-4.505557e+02,3.649232e+03
3,POS_0004,MSFT,223,equity,NaN,NaN,NaN,456.66,0.80,19.42,...,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.230000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,POS_0005,AMZN,-14,equity,NaN,NaN,NaN,238.18,0.00,21.77,...,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.400000e+01,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
5,POS_0006,TSLA,65,option,446.25,2026-05-03,put,438.57,0.00,26.69,...,-9.960472e-01,1.013666e-02,2.770384e+00,1.565987e+01,-1.277580e+02,-6.474307e+01,6.588829e-01,1.800750e+02,1.017891e+03,-8.304273e+03
6,POS_0007,TSLA,-44,option,521.90,2026-09-28,put,438.57,0.00,26.69,...,-1.000000e+00,7.116778e-51,1.132132e-47,1.814983e+01,-3.542860e+02,4.400000e+01,-3.131382e-49,-4.981380e-46,-7.985927e+02,1.558858e+04
7,POS_0008,TSLA,313,equity,NaN,NaN,NaN,438.57,0.00,26.69,...,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.130000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
8,POS_0009,GOOG,-946,equity,NaN,NaN,NaN,333.16,0.25,16.01,...,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-9.460000e+02,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
9,POS_0010,AAPL,19,option,306.60,2026-07-16,put,258.21,0.40,210.00,...,-1.000000e+00,0.000000e+00,0.000000e+00,1.073981e+01,-1.485650e+02,-1.900000e+01,0.000000e+00,0.000000e+00,2.040563e+02,-2.822735e+03
